In [2]:
%run file_paths_updt Copy

StatementMeta(, 77426ae0-30ad-4ce1-aa04-a1a8521a5dda, 3, Finished, Available, Finished)

In [ ]:
%run file_paths_updt Copy

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import sys

def brnze_silver():
    try:
        df_new = spark.read.parquet(Bronze+'//'+'hr_data_bronze.parquet')
        filtered = [i for i in df_new.columns if i!='termdate'] 
        df = df_new.withColumn('hire_date',to_date(col("hire_date"), "M/d/yyyy"))\
                   .withColumn('birthdate',to_date(col("birthdate"),"M/d/yyyy"))\
                   .withColumn('employee_id',regexp_replace(col('id'),'-',''))
        pcs_df = df.dropDuplicates().dropna(subset=filtered)
        dfnew = pcs_df.withColumn('emp_id_int',row_number().over(Window.orderBy('employee_id')))
        dfnew.write.format("delta").mode('overwrite').save(Silver+'//'+'hr_data_silver')
    except:
        print(sys.exc_info())

brnze_silver()

# The brnze_silver() function reads the Parquet file from the Bronze layer, cleans the data by formatting dates, 
# removing duplicates, handling nulls (except for termdate), and fixing employee IDs. It then adds a unique integer 
# ID (emp_id_int) to each employee using a row number window function and saves the cleaned data in Delta format to the Silver layer.
#  Errors are caught and printed if they occur.

StatementMeta(, 77426ae0-30ad-4ce1-aa04-a1a8521a5dda, 7, Finished, Available, Finished)